Note: __yfinance can be unstable/unreliable from time to time__ when it comes to downloading fundamental data with the ticker object. In particular, __ticker.get_info()__ can be flawed.

__Action required__: Check for the __latest yfinance versions__ and update with the following command (Anaconda Prompt / Terminal):

pip install yfinance --upgrade

In the following, I have added an __alternative Yahoo Finance API Wrapper__, __yahooquery__

__Action required: Please install yahooquery with the following command (Anaconda Prompt / Terminal):__

pip install yahooquery --upgrade

Note: 
- Please make sure you have installed the __latest__ yahooquery version (if not, please upgrade). 

__Update December 2023__: Sometimes yfinance is more stable/reliable and sometimes yahooquery is more stable/reliable. Most recently, yfinance seemed to be better. In any case, __please make sure you have the latest versions installed!__

# Keystone Project - The Dow Jones Index Constituents

__Assignment / Goal:__

1. Load __historical price and volume data__ for all __30 stocks/constituents__ of the Dow Jones Industrial Average (__DJIA__) stock index.

2. Load and save other stock information like __Exchange, Industry, Price Performance, Dividend Yield, Price-to-Book ratio and Forward P/E ratio__.

3. __Compare/sort stocks__ by Price Performance, Dividend Yield, Price-to-Book ratio and Forward P/E ratio.

What is the Dow Jones Industrial Average (DJIA)?

__"The Dow Jones Industrial Average (DJIA), Dow Jones, or simply the Dow, is a stock market index of 30 prominent companies listed on stock exchanges in the United States.[...]"__ (Wikipedia) https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average

(hint: you can load tables from a website with pd.read_html(url))

### ------------------------SOLUTION------------------------------

## Loading the DJIA 30 Constituents from the Web

In [ ]:
import yfinance as yf
from yahooquery import Ticker
import pandas as pd

In [ ]:
url = "https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average"
url

In [ ]:
df = pd.read_html(url)[2] # now [2]
df

In [ ]:
df.info()

In [ ]:
df.rename(columns = {"Date added":"Date_Added"}, inplace = True)
df.rename(columns = {"Index weighting":"Weights"}, inplace = True)

In [ ]:
df.Date_Added = pd.to_datetime(df.Date_Added)

In [ ]:
df.Weights = pd.to_numeric(df.Weights.str.replace("%", ""))

In [ ]:
df

In [ ]:
df.Weights.sum()

In [ ]:
df.drop(columns = "Notes", inplace = True)

In [ ]:
df.set_index("Symbol", inplace = True)

In [ ]:
df

In [ ]:
symbols = df.index.to_list()
symbols

In [ ]:
last_update = df["Date_Added"].max()
last_update

## Load Historical Price/Volume Data (Time Series)

In [ ]:
symbols

In [ ]:
last_update

In [ ]:
ts = yf.download(tickers = symbols, start = last_update)
ts

In [ ]:
ts.info()

In [ ]:
ts.Close

In [ ]:
ts.dropna(inplace = True)

In [ ]:
perf = ts.Close.iloc[-1].div(ts.Close.iloc[0]).sub(1).sort_values(ascending = False)
perf # total price increase/decrease (in %) since last reconstitution

In [ ]:
perf.index.name = "Symbol"

In [ ]:
perf

In [ ]:
df

In [ ]:
df["Performance"] = perf
df

In [ ]:
df.sort_values(by = "Performance", ascending = False)

## More detailed Stock Data (cross-sectional)

__yfinance__ (recommended, much faster than yahooquery)

In [ ]:
msft = yf.Ticker(ticker = "msft") 
msft

In [ ]:
msft.get_info()

In [ ]:
info = pd.Series(msft.get_info()).to_frame().T
info

In [ ]:
symbols

In [ ]:
cs = pd.DataFrame()
cs

In [ ]:
count = 1
for symbol in symbols:
    try:
        info = pd.Series(yf.Ticker(ticker = symbol).get_info()).to_frame().T
        cs = pd.concat([cs, info])
        print(count, end = '\r')
        count += 1
    except Exception as e:
        print("{} not found".format(symbol))
print("Download complete.")
cs["symbol"] = symbols # Added

In [ ]:
cs

In [ ]:
cs.set_index("symbol", inplace = True)

In [ ]:
cs.index.name = "Symbol"
cs

In [ ]:
cs.info()

__yahooquery__ (slow, better use yfinance)

In [ ]:
msft = Ticker("MSFT")
msft

In [ ]:
msft.key_stats["MSFT"] # key statistics

In [ ]:
msft.financial_data["MSFT"] # financial data

In [ ]:
msft.summary_detail["MSFT"] # summary statistics

In [ ]:
symbols

In [ ]:
cs = pd.DataFrame()
cs

In [ ]:
count = 1
for symbol in symbols:
    try:
        all_info = Ticker(symbol).key_stats[symbol] # key statistics
        all_info.update(Ticker(symbol).financial_data[symbol]) # financial data
        all_info.update(Ticker(symbol).summary_detail[symbol]) # summary_details
        all_info = pd.Series(all_info).to_frame().T
        cs = pd.concat([cs, all_info])
        print(count, end = '\r')
        count += 1
    except Exception as e:
        print("{} not found".format(symbol))
print("Download complete.")
cs["symbol"] = symbols # Added

In [ ]:
cs

In [ ]:
cs.set_index("symbol", inplace = True)
cs

In [ ]:
cs.index.name = "Symbol"
cs

In [ ]:
cs.info()

## Equity Analysis & Comparison

In [ ]:
df

In [ ]:
cs

__Price-to-Book ratio__ 

In [ ]:
cs["priceToBook"].sort_values(ascending = False)

In [ ]:
cs.loc["BA", "bookValue"]

__-> negative/close to zero Book Value of Equity distorts P/B ratio__

In [ ]:
df["PricetoBook"] = (cs.currentPrice / cs.bookValue)
df

__Dividend Yield__ (Dividend per Share / Price)

In [ ]:
cs.dividendYield

In [ ]:
df["Dividend_Yield"] = cs.dividendYield.fillna(0)
df

__Forward Price-to-Earnings Ratio__

In [ ]:
df["ForwardPE"] = cs.forwardPE
df

__Comparison__

In [ ]:
df.sort_values("PricetoBook", ascending = False)

In [ ]:
df.sort_values("ForwardPE", ascending = False)

In [ ]:
df.sort_values("Dividend_Yield", ascending = False)

## Keystone Project - Extended

Repeat the Keystone Project for __other Stock Indexes / Stock Markets__ of your choice. Major Challenge: Find __Ticker Symbol List__ on the Web

### Examples:

__Full US Market__

https://www.nasdaq.com/market-activity/stocks/screener

__S&P 500__

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies

__German DAX 40__

https://en.wikipedia.org/wiki/DAX

__EURO STOXX 50__

https://en.wikipedia.org/wiki/EURO_STOXX_50

__FTSE 100__

https://en.wikipedia.org/wiki/FTSE_100_Index

__Listings for worldwide exchanges__

https://www.interactivebrokers.com/en/index.php?f=1562&p=north_america

__Example: India__

In [ ]:
import pandas as pd

In [ ]:
url = "https://www.interactivebrokers.com/en/index.php?f=2222&exch=nse&showcategories=STK&p=&cc=&limit=100&page={}"

In [ ]:
pd.read_html(url.format(1))[2]

In [ ]:
pages = 22

In [ ]:
df = pd.DataFrame()

In [ ]:
for i in range(1, pages + 1):
    table = pd.read_html(url.format(i))[2]
    df = pd.concat([df, table])

In [ ]:
df

In [ ]:
df.reset_index(drop = True, inplace = True)

In [ ]:
df

In [ ]:
df.info()

__Reminder__: to pull data for an [Indian] Stock from yfinance, you need to __add the exchange-specific Suffix__ [.NS]
https://help.yahoo.com/kb/SLN2310.html

In [ ]:
import yfinance as yf

In [ ]:
symbol = df.Symbol[0] + ".NS"
symbol

In [ ]:
yf.download(symbol)